In [ ]:
######CIRCLE TRAJECTORY########

#Simulation parameters
T_HORIZON = 20.0
DT=0.05
SIM_HORIZON = int(T_HORIZON/DT)

#control parameters
vMax = 1.5
vMin = 0.2
angleMax = np.pi * (1/7.2)

# trajectory initialization
x0 = np.array([0.0, 0.0, 0.0])

#hisotry
x_t = np.zeros((SIM_HORIZON, 3))
u_t = np.zeros((SIM_HORIZON, 2))


x_t[0]=x0
u=np.array([0.0, 0.0])

for i in range(0, SIM_HORIZON-1):

    u[0]=vMin
    u[1]=angleMax
    
    #propagate the system using the filtered control and save history
    x_t[i+1] = TVD_rk3(x_t[i], rc3D_dyn, u, DT)
    u_t[i] = u


# Extract x and y coordinates
x_coords = x_t[:, 0]
y_coords = x_t[:, 1]

# Create scatter plot
plt.scatter(x_coords, y_coords, c='blue', marker='o')
plt.title('Trajectory of RC Car')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.grid(True)
plt.show()

# print the maximum x and y values


In [ ]:
####Backup plot section as ipywidgets breaks sometimes####

idx_to_plot = 0

#def plot_func(idx_to_plot):
    
state_plot = x_t[idx_to_plot]
#print the state with 2 decimals
print('[x,y,th]')
print(np.around(state_plot, decimals=2))
#get the index of the closest v and th to the state_first_fltr
th_idx = np.argmin(np.abs(matlab_var_dict['th_coord'] - state_plot[2]))

f1, ax1 = plt.subplots()
data1 = data_lx
data1 = data1.transpose()
CP1 = ax1.contour(matlab_var_dict['x_coord'],matlab_var_dict['y_coord'],data1,0)
for c in CP1.collections:
    c.set_edgecolor("red")
    c.set_linewidth(1)
    c.set_label("")

data2 = data[:,:,th_idx] #slice of lx
data2 = data2.transpose()
CP2 = ax1.contour(matlab_var_dict['x_coord'],matlab_var_dict['y_coord'],data2,0)
for c in CP2.collections:
    c.set_edgecolor("blue")
    c.set_linewidth(1)
    c.set_label("")

#plot an arrow with angle theta
th=matlab_var_dict['th_coord']
th = th[th_idx]

SP1 = ax1.scatter(x_t[0:idx_to_plot,0], x_t[0:idx_to_plot,1],s=5,c=m_t[0:idx_to_plot], alpha=0.5)
SP2 = ax1.scatter(x_t[idx_to_plot,0], x_t[idx_to_plot,1],s=10, c=[[0.0,0.0,0.0]], alpha=1.0)
#plot the arrow
ax1.arrow(x_t[idx_to_plot,0], x_t[idx_to_plot,1], np.cos(th)*0.2, np.sin(th)*0.2, head_width=0.1, head_length=0.1, fc='k', ec='k')

#plot the hallucinations corresponding to idx_to_plot
for h in list_hallucinations_at_idx[idx_to_plot]:
    #remove zero rows
    h = h[~np.all(h == 0, axis=1)]
    ax1.plot(h[:,0], h[:,1], color='green', alpha=0.2)

#set x and y limits
ax1.set_xlim([0, 6])
ax1.set_ylim([0, 4])

#change aspect ratio to match the grid
ax1.set_aspect('equal')
plt.show()

#interact(plot_func, idx_to_plot=(0, i, 1))